In [1]:
from pathlib import Path

signal_images = Path.cwd().parent / "serial2p_subset" / "ch02"
background_images = Path.cwd().parent / "serial2p_subset" / "ch00"
voxel_sizes = [5, 2, 2] # in microns

In [ ]:
import pooch
data_path = Path.cwd().parent / "serial_2p_subset"
# Use pooch to fetch data if it hasn't already been downloaded
dsb_data_url = "https://gin.g-node.org/BrainGlobe/demo-materials/raw/master/serial2p_subset.zip"
data_path = pooch.retrieve(dsb_data_url, known_hash=None, progressbar=True, processor=pooch.Unzip(extract_dir=data_path))
data_path = Path(data_path[0]).parent.parent.parent


 27%|█████████▎                        | 1.52G/5.59G [1:01:00<2:57:30, 382kB/s]

In [13]:
import glob
import os

from dask import array as da
from dask import delayed
from imlib.general.system import get_sorted_file_paths
from tifffile import TiffFile, imread

In [14]:
# Write a function to load data with dask
# Alternatively - `from cellfinder_core.tools.IO import read_with_dask`

lazy_imread = delayed(imread) 

def get_tiff_meta(path):
    with TiffFile(path) as tfile:
        nz = len(tfile.pages)
        if not nz:
            raise ValueError(f"tiff file {path} has no pages!")
        first_page = tfile.pages[0]

    return tfile.pages[0].shape, first_page.dtype


def read_with_dask(path):
    filenames = glob.glob(os.path.join(path, "*.tif"))
    shape, dtype = get_tiff_meta(filenames[0])
    lazy_arrays = [lazy_imread(fn) for fn in get_sorted_file_paths(filenames)]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=shape, dtype=dtype)
        for delayed_reader in lazy_arrays
    ]
    stack = da.stack(dask_arrays, axis=0)
    return stack

In [17]:
from cellfinder_core.main import main as cellfinder_run

signal_array = read_with_dask(signal_images)
background_array = read_with_dask(background_images)

print(signal_array)
print(background_array)

dask.array<stack, shape=(100, 3868, 5416), dtype=uint16, chunksize=(1, 3868, 5416), chunktype=numpy.ndarray>
dask.array<stack, shape=(100, 3868, 5416), dtype=uint16, chunksize=(1, 3868, 5416), chunktype=numpy.ndarray>


In [16]:
detected_cells = cellfinder_run(signal_array,background_array,voxel_sizes)

Processing planes:   1%|██▌                                                                                                                                                                                                                                                         | 1/100 [00:10<16:35, 10.05s/it]

Plane 0 received for 3D filtering
Plane 1 received for 3D filtering
Plane 2 received for 3D filtering


Processing planes:   3%|███████▌                                                                                                                                                                                                                                                    | 3/100 [00:11<04:57,  3.07s/it]

Plane 3 received for 3D filtering


Processing planes:   4%|██████████                                                                                                                                                                                                                                                  | 4/100 [00:12<03:49,  2.39s/it]

...

Plane 98 received for 3D filtering


Processing planes:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 99/100 [01:14<00:00,  1.71it/s]

Plane 99 received for 3D filtering


Processing planes: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:14<00:00,  1.34it/s]


Detection complete - all planes done in : 0:01:14.977864
87/87 [==============================] - 40s 437ms/step
